In [ ]:
%load_ext blackcellmagic

#### Imports and setup

In [ ]:
import numpy as np
import pandas as pd
import deepdish as dd
import h5py
import nibabel.freesurfer.mghformat as mgh
import matplotlib.pyplot as plt
import seaborn as sns

from statsmodels.stats.anova import AnovaRM

In [ ]:
from spacestream.core.constants import SUBJECTS, CORE_ROI_NAMES
from spacestream.core.paths import DATA_PATH, NSDDATA_PATH, RESULTS_PATH
from spacestream.utils.general_utils import sem
from spacestream.utils.get_utils import get_mapping

In [ ]:
seeds = [0,1,2,3,4]
hemis = ["lh","rh"]
sw_list = [
            "0.0",
            "0.1",
            "0.25",
            "0.5",
            "1.25",
            "2.5",
            "25.0",
        ]
sup_case = ["simCLR", "supervised"]
colors = {}
colors["simCLR"], colors["supervised"] = "#720298", "#B59410"

#### Load and format data

In [ ]:
def reject_outliers(data, m=3.):
    d = np.abs(data - np.nanmean(data))
    mdev = np.nanmean(d)
    s = d / (mdev if mdev else 1.)
    return data[s < m]

In [ ]:
all_nc = [
    [[[] for j in range(len(CORE_ROI_NAMES))] for i in range(len(SUBJECTS))]
    for h in range(len(hemis))
]
all_s2s_rs = [
    [[[] for j in range(len(CORE_ROI_NAMES))] for i in range(len(SUBJECTS))]
    for h in range(len(hemis))
]
all_rs = [
    [
        [
            [[[] for m in range(len(sw_list))] for s in range(len(sup_case))]
            for j in range(len(CORE_ROI_NAMES))
        ]
        for i in range(len(SUBJECTS))
    ]
    for h in range(len(hemis))
]


for hidx, hemi in enumerate(hemis):
    for sidx, sid in enumerate(SUBJECTS):

        # get ROI info
        mgh_file = mgh.load(
            NSDDATA_PATH
            + "nsddata/freesurfer/subj"
            + sid
            + "/label/"
            + hemi
            + ".streams_shrink10.mgz"
        )
        streams = mgh_file.get_fdata()[:, 0, 0]
        streams_trim = streams[streams != 0]
        # get noise ceiling estimates
        mgh_file = mgh.load(
            NSDDATA_PATH
            + "/nsddata_betas/ppdata/subj"
            + sid
            + "/nativesurface/betas_fithrf_GLMdenoise_RR/"
            + hemi
            + ".nc_3trials.mgh"
        )
        NC = mgh_file.get_fdata()[:, 0, 0]
        NC_trim = NC[streams != 0]
        NC_trim[NC_trim == 0] = np.nan  # Set all 0s to nans to avoid dividing by 0

        for r in range(len(CORE_ROI_NAMES)):
            all_nc[hidx][sidx][r] = NC_trim[streams_trim == r + 5] / 100

        # Get subject 2 subject mapping estimates
        load_path = (
            RESULTS_PATH
            + "mappings/regression/brain2brain/subj"
            + sid
            + "_"
            + hemi
            + "_streams_shrink10_to_streams_shrink10_Ridge_1CV_10splits__subsample_10000voxels_subsamptype1_pooled1.hdf5"
        )
        rsquared_array = {}
        with h5py.File(load_path, "r") as f:
            keys = f.keys()
            for k in keys:
                rsquared_array[k] = f[k][:]
        for r, roin in enumerate(
            CORE_ROI_NAMES
        ):  # use only the corresponding ROI as source results
            if roin == "Dorsal":
                roin = "Parietal" #old naming convention
            all_s2s_rs[hidx][sidx][r] = rsquared_array[roin][streams_trim == r + 5]

        for supidx, sup in enumerate(sup_case):
            for swidx, sw_name in enumerate(sw_list):

                if sup == "simCLR":
                    model_name = "spacetorch"
                else:
                    model_name = "spacetorch_supervised"

                # get model fits
                load_path = (
                    RESULTS_PATH
                    + "mappings/regression/model2brain/subj"
                    + sid
                    + "_"
                    + hemi
                    + "_streams_shrink10_"
                    + model_name
                    + sw_name
                    + "_base_model.layer4.1_Ridge_subsample_2_1CV_1pretraining_fits.hdf5"
                )

                fits = {}
                with h5py.File(load_path, "r") as f:
                    keys = f.keys()
                    for k in keys:
                        fits[k] = f[k][:]

                max_rs = fits["base_model.layer4.1"]
                for r in range(len(CORE_ROI_NAMES)):
                    all_rs[hidx][sidx][r][supidx][swidx] = max_rs[streams_trim == r + 5]

In [ ]:
corrected_means = np.empty(
    (len(hemis), len(SUBJECTS), len(CORE_ROI_NAMES), len(sup_case), len(sw_list))
)
normed_s2s = np.empty((len(hemis), len(SUBJECTS), len(CORE_ROI_NAMES)))

for hidx, hemi in enumerate(hemis):
    for sidx, sid in enumerate(SUBJECTS):

        normed_s2s[hidx, sidx] = [
            np.nanmean(
                reject_outliers(all_s2s_rs[hidx][sidx][r] / all_nc[hidx][sidx][r])
            )
            for r in range(len(CORE_ROI_NAMES))
        ]

        for supidx, model_name in enumerate(sup_case):

            for midx, sw_name in enumerate(sw_list):

                for r in range(len(CORE_ROI_NAMES)):
                    corrected_means[hidx][sidx][r][supidx][midx] = np.nanmean(
                        reject_outliers(
                            all_rs[hidx][sidx][r][supidx][midx] / all_nc[hidx][sidx][r]
                        )
                    )
                    
#average across hemis
total_means = np.mean(corrected_means, axis=0)

In [ ]:
dataframes = {}

for ridx, r in enumerate(CORE_ROI_NAMES):
    print(ridx)

    d = {
        "subject_name": np.concatenate(
            (
                np.tile("subj01", len(sw_list) * len(sup_case)),
                np.tile("subj02", len(sw_list) * len(sup_case)),
                np.tile("subj03", len(sw_list) * len(sup_case)),
                np.tile("subj04", len(sw_list) * len(sup_case)),
                np.tile("subj05", len(sw_list) * len(sup_case)),
                np.tile("subj06", len(sw_list) * len(sup_case)),
                np.tile("subj07", len(sw_list) * len(sup_case)),
                np.tile("subj08", len(sw_list) * len(sup_case)),
            )
        ),
        "spatial_weight": np.tile([float(sw) for sw in sw_list], len(SUBJECTS) * len(sup_case)),
        "version": np.ravel(
            np.tile(
                np.ravel([np.tile(m, len(sw_list)) for m in sup_case]),
                len(SUBJECTS),
            )
        ),
        "means": (total_means[:, ridx, :]).ravel(),
    }
    dataframes[r] = pd.DataFrame(
        d, columns=["subject_name", "spatial_weight", "version", "means"]
    )

#### Plotting

In [ ]:
sns.set_theme(style="ticks")
line_kwargs = {"marker": ".", "markersize": 25, "lw": 1, 'markeredgecolor': 'w', 'markeredgewidth': 2}

for ridx, stream in enumerate(CORE_ROI_NAMES):
    fig, ax = plt.subplots(figsize = (3,5))

    sns.lineplot(
        data=dataframes[stream],
        x="spatial_weight",
        y="means",
        hue="version",
        palette=[colors["simCLR"], colors["supervised"]],
        errorbar='se',
        **line_kwargs
    )
    ax.set_xscale("symlog", linthresh=0.09)
    ax.set_xlim([-0.01, 50])
    ax.set_xticks([], minor=True)
    ax.set_xticks([0, 0.1, 0.25, 0.5, 1.25, 2.5, 25])
    ax.set_xticklabels([0, 0.1, "", "", 1.25, "", 25])
    
    xmin = 0
    xmax = 1
    vals = normed_s2s[:,ridx]
    y1 = np.mean(vals) - np.std(vals)
    y2 = np.mean(vals) + np.std(vals)
    ax.axhspan(y1, y2, xmin=0, xmax=1, color="lightgray", alpha=0.75)
    
    ax.legend([],[], frameon=False)

    ax.set_xlabel("Spatial Weight", fontsize=20)
    ax.set_ylabel("Correlation", fontsize=20)
    plt.ylim(bottom=0.0, top=0.8)

    ax.set_yticks([])
    ax.spines["left"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.spines["top"].set_visible(False)


    plt.show()

    save_path = RESULTS_PATH + "/figures/3d_" + stream + "_" + "_despined_SD.png"
    fig.savefig(save_path, dpi=100)

#### Statistics

In [ ]:
aovrm = AnovaRM(dataframes["Dorsal"],'means', 'subject_name', within=['spatial_weight','version'], aggregate_func='mean')
res = aovrm.fit()
print(res.anova_table.to_latex())

In [ ]:
aovrm = AnovaRM(dataframes["Lateral"],'means', 'subject_name', within=['spatial_weight','version'], aggregate_func='mean')
res = aovrm.fit()
print(res.anova_table.to_latex())

In [ ]:
aovrm = AnovaRM(dataframes["Ventral"],'means', 'subject_name', within=['spatial_weight','version'], aggregate_func='mean')
res = aovrm.fit()
print(res.anova_table.to_latex())

In [ ]:
v0 = dataframes['Ventral'][dataframes['Ventral']['spatial_weight']==0.0]
v0

In [ ]:
np.mean(v0[v0['version']=="simCLR"]['means'])

In [ ]:
np.std(v0[v0['version']=="simCLR"]['means'])

In [ ]:
np.mean(v0[v0['version']=="supervised"]['means'])

In [ ]:
np.std(v0[v0['version']=="supervised"]['means'])